# Chapter 2 : Image Clasification with Pytorch

## Making outr first Neural Network in PyTorch 

In [11]:
# imports
import torch, torchvision
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F

from torchvision import transforms
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES=True

In [26]:
# checking for GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

## Importing Data

In [32]:
train_data_path = "./train/"
val_data_path = "./val/"
test_data_path = "./test/"

# creating transformation pipeline
transformation = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [33]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

In [34]:
# getting datasets
train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=transformation, is_valid_file=check_image)
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=transformation, is_valid_file=check_image)
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transformation, is_valid_file=check_image)

In [35]:
# building Data Loaders
batch_size=128
train_data_loader = data.DataLoader(train_data, batch_size=batch_size)
val_data_loader = data.DataLoader(val_data, batch_size=batch_size)
test_data_loader = data.DataLoader(test_data, batch_size=batch_size)

## Creating Model

In [41]:
# Creating Networf
class SimpleNet(nn.Module):
    
    # Initializing Layers and variables
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50, 2)

    # forward function
    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [42]:
# initiating model
simplenet = SimpleNet()

# adding model to GPU
simplenet.to(device)

SimpleNet(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

In [43]:
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

## Training

In [55]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(1, epochs+1):
        
        model.train()
        train_loss = 0.0

        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch

            inputs = inputs.to(device)
            targets = targets.to(device)

            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.data.item()

        train_loss /= len(train_loader.dataset)

        model.eval()
        val_loss = 0.0
        
        num_correct = 0
        num_examples = 0

        for batch in val_loader:
            # NO : optimizer.zero_grad()
            inputs, targets = batch

            inputs = inputs.to(device)
            targets = targets.to(device)

            output = model(inputs)
            loss = loss_fn(output, targets)

            val_loss += loss.data.item() * inputs.size(0)
            
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0] 

            # loss.backward()
            # optimizer.step()
            # train_loss += loss.data.item()

        val_loss /= len(val_loader.dataset)

        # printing statememnt per epoch
        print(f"Epoch: {epoch}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.3f}, Accuracy: {(num_correct/num_examples):.2f}")


In [57]:
# Calling Training Function
train(simplenet, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, val_data_loader, epochs=3, device=device)

Epoch: 1, Training Loss: 0.0000, Validation Loss: 1.177, Accuracy: 0.72
Epoch: 2, Training Loss: 0.0000, Validation Loss: 1.272, Accuracy: 0.71
Epoch: 3, Training Loss: 0.0000, Validation Loss: 1.261, Accuracy: 0.71


## Inference

In [58]:
labels = ['cat','fish']

img = Image.open("./val/fish/100_1422.JPG") 
img = transformation(img).to(device)
img = torch.unsqueeze(img, 0)

simplenet.eval()
prediction = F.softmax(simplenet(img), dim=1)
prediction = prediction.argmax()
print(labels[prediction]) 

fish


In [59]:
labels = ['cat','fish']

for batch in test_data_loader:
    inputs, targets = batch
    inputs = inputs.to(device)
    targets = targets.to(device)

    predictions = simplenet(inputs)

    for i, prediction in enumerate(predictions):
        prediction = prediction.argmax()
        print(f"#{i+1} === Prediction: {labels[prediction]}, Actual Label: {labels[targets[i]]}")

#1 === Prediction: cat, Actual Label: cat
#2 === Prediction: cat, Actual Label: cat
#3 === Prediction: cat, Actual Label: cat
#4 === Prediction: cat, Actual Label: cat
#5 === Prediction: cat, Actual Label: cat
#6 === Prediction: fish, Actual Label: cat
#7 === Prediction: cat, Actual Label: cat
#8 === Prediction: cat, Actual Label: cat
#9 === Prediction: cat, Actual Label: cat
#10 === Prediction: cat, Actual Label: cat
#11 === Prediction: cat, Actual Label: cat
#12 === Prediction: cat, Actual Label: cat
#13 === Prediction: cat, Actual Label: cat
#14 === Prediction: cat, Actual Label: cat
#15 === Prediction: cat, Actual Label: cat
#16 === Prediction: cat, Actual Label: cat
#17 === Prediction: fish, Actual Label: cat
#18 === Prediction: cat, Actual Label: cat
#19 === Prediction: cat, Actual Label: cat
#20 === Prediction: cat, Actual Label: cat
#21 === Prediction: cat, Actual Label: cat
#22 === Prediction: cat, Actual Label: cat
#23 === Prediction: cat, Actual Label: cat
#24 === Prediction

In [ ]:
labels = ['cat','fish']
correct = 0
total = 0

with torch.no_grad():
    for batch in test_data_loader:
        inputs, targets = batch
        inputs = inputs.to(device)
        targets = targets.to(device)

        predictions = simplenet(inputs)

        for i, prediction in enumerate(predictions):
            prediction = prediction.argmax()
            if prediction == targets[i]:
                correct += 1
            total += 1
        
        accuracy = correct / total
        print(f"Accuracy of this Batch {i+1} : {accuracy}")

## Saving & Loading Model

In [62]:
torch.save(simplenet, "./model/simplenet")

In [63]:
simplenet = torch.load("./model/simplenet")

In [64]:
torch.save(simplenet.state_dict(), "./model/simplenet-state-dict")

In [65]:
simplenet = SimpleNet()
simplenet_state_dict = torch.load("./model/simplenet-state-dict")
simplenet.load_state_dict(simplenet_state_dict)

<All keys matched successfully>